In [1]:
import tensorflow as tf
import pandas as pd
import datetime
from pandas_profiling import ProfileReport
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers


In [2]:
# Read movement data, this could be from several locations like S3 buckets
movements = pd.read_csv("../data/SmartMovementExport.csv")

In [3]:
movements.head()

,id,acceloX,acceloY,acceloZ,userAcceloX,userAcceloY,userAcceloZ,gyroX,gyroY,gyroZ,lightSensor,locked
0,2020-11-14 22:18:31.510956,-2.109356,2.882707,9.466963,0.218186,0.537032,0.203292,-0.711324,-0.419993,0.339640,8,0
1,2020-11-14 22:18:31.558048,-2.109356,2.882707,9.466963,0.676335,0.776491,0.463521,-0.711324,-0.419993,0.339640,8,0
2,2020-11-14 22:18:31.558393,-2.109356,2.882707,9.466963,0.676335,0.776491,0.463521,-1.693300,-1.869679,0.440446,8,0
3,2020-11-14 22:18:31.558616,-3.510007,1.884294,9.316123,0.676335,0.776491,0.463521,-1.693300,-1.869679,0.440446,8,0
4,2020-11-14 22:18:31.669076,-3.510007,1.884294,9.316123,0.676335,0.776491,0.463521,-1.693300,-1.869679,0.440446,7,0


In [4]:
CSV_COLUMN_NAMES = ['id', 'acceloX', 'acceloY', 'acceloZ', 'userAcceloX', 'userAcceloY','userAcceloZ', 'gyroX', 'gyroY', 'gyroZ', 'lightSensor', 'locked']

In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(movements, movements['locked'], test_size=0.33, random_state=42)

Explore the smart movement data with PandasProfiling

In [6]:
profile = ProfileReport(movements, title="Pandas Profiling Report")

In [7]:
profile.to_widgets()


Generate report structure: 100%|██████████| 1/1 [00:03<00:00,  3.20s/it]


Create training and validation set

In [8]:
val_dataframe = movements.sample(frac=0.2, random_state=1337)
train_dataframe = movements.drop(val_dataframe.index)
print(
    "Using %d samples for training and %d for validation"
    % (len(train_dataframe), len(val_dataframe))
)

Using 2559 samples for training and 640 for validation


Let's generate `tf.data.Dataset` objects for each dataframe:

In [9]:
def dataframe_to_dataset(dataframe):
    dataframe = dataframe.copy()
    labels = dataframe.pop("locked")
    ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
    ds = ds.shuffle(buffer_size=len(dataframe))
    return ds

train_ds = dataframe_to_dataset(train_dataframe)
val_ds = dataframe_to_dataset(val_dataframe)

In [10]:

for x, y in train_ds.take(1):
    print("Input:", x)
    print("Target:", y)

Input: {'id': <tf.Tensor: shape=(), dtype=string, numpy=b'2020-11-14 22:19:00.694162'>, 'acceloX': <tf.Tensor: shape=(), dtype=float64, numpy=-3.0096035003662114>, 'acceloY': <tf.Tensor: shape=(), dtype=float64, numpy=7.738296031951903>, 'acceloZ': <tf.Tensor: shape=(), dtype=float64, numpy=6.8931169509887695>, 'userAcceloX': <tf.Tensor: shape=(), dtype=float64, numpy=-0.05670356750488281>, 'userAcceloY': <tf.Tensor: shape=(), dtype=float64, numpy=1.340930461883545>, 'userAcceloZ': <tf.Tensor: shape=(), dtype=float64, numpy=0.5722548961639404>, 'gyroX': <tf.Tensor: shape=(), dtype=float64, numpy=0.21089068055152893>, 'gyroY': <tf.Tensor: shape=(), dtype=float64, numpy=-0.27607011795043945>, 'gyroZ': <tf.Tensor: shape=(), dtype=float64, numpy=-0.03291177749633789>, 'lightSensor': <tf.Tensor: shape=(), dtype=int64, numpy=12>}
Target: tf.Tensor(0, shape=(), dtype=int64)


Batch the datasets

In [11]:
train_ds = train_ds.batch(32)
val_ds = val_ds.batch(32)

Now build a model

In [12]:
# Numerical inout layer
acceloX = keras.Input(shape=(1,), name="acceloX")
acceloY = keras.Input(shape=(1,), name="acceloY")
acceloZ = keras.Input(shape=(1,), name="acceloZ")
userAcceloX = keras.Input(shape=(1,), name="userAcceloX")
userAcceloY = keras.Input(shape=(1,), name="userAcceloY")
userAcceloZ = keras.Input(shape=(1,), name="userAcceloZ")
gyroX = keras.Input(shape=(1,), name="gyroX")
gyroY = keras.Input(shape=(1,), name="gyroY")
gyroZ = keras.Input(shape=(1,), name="gyroZ")
lightSensor = keras.Input(shape=(1,), name="lightSensor")
all_inputs = [

        acceloX,
        acceloY,
        acceloZ,
        userAcceloX,
        userAcceloY,
        userAcceloZ,
        gyroX,
        gyroY,
        gyroZ,
        lightSensor
    ]


In [13]:
from tensorflow.keras.layers.experimental.preprocessing import Normalization
def encode_numerical_feature(feature, name, dataset):
    # Create a Normalization layer for our feature
    normalizer = Normalization()

    # Prepare a Dataset that only yields our feature
    feature_ds = dataset.map(lambda x, y: x[name])
    feature_ds = feature_ds.map(lambda x: tf.expand_dims(x, -1))

    # Learn the statistics of the data
    normalizer.adapt(feature_ds)

    # Normalize the input feature
    encoded_feature = normalizer(feature)
    return encoded_feature

In [14]:

acceloX_encoded = encode_numerical_feature(acceloX, "acceloX", train_ds)
acceloY_encoded = encode_numerical_feature(acceloY, "acceloY", train_ds)
acceloZ_encoded = encode_numerical_feature(acceloZ, "acceloZ", train_ds)
userAcceloX_encoded = encode_numerical_feature(userAcceloX, "userAcceloX", train_ds)
userAcceloY_encoded = encode_numerical_feature(userAcceloY, "userAcceloY", train_ds)
userAcceloZ_encoded = encode_numerical_feature(userAcceloZ, "userAcceloZ", train_ds)
gyroX_encoded = encode_numerical_feature(gyroX, "gyroX", train_ds)
gyroY_encoded = encode_numerical_feature(gyroY, "gyroY", train_ds)
gyroZ_encoded = encode_numerical_feature(gyroZ, "gyroZ", train_ds)
lightSensor_encoded = encode_numerical_feature(lightSensor, "lightSensor", train_ds)

all_features = layers.concatenate(
    [
        acceloX_encoded,
        acceloY_encoded,
        acceloZ_encoded,
        userAcceloX_encoded,
        userAcceloY_encoded,
        userAcceloZ_encoded,
        gyroX_encoded,
        gyroY_encoded,
        gyroZ_encoded,
        lightSensor_encoded
    ]
)

In [15]:
x = layers.Dense(32, activation="relu")(all_features)
x = layers.Dropout(0.5)(x)
output = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(all_inputs, output)
model.compile("adam", "binary_crossentropy", metrics=["accuracy"])

In [16]:
keras.utils.plot_model(model, show_shapes=True, rankdir="LR")


('Failed to import pydot. You must `pip install pydot` and install graphviz (https://graphviz.gitlab.io/download/), ', 'for `pydotprint` to work.')


## Train the model

In [17]:
model.fit(train_ds, epochs=50, validation_data=val_ds)

Epoch 1/50
80/80 [==============================] - 0s 4ms/step - loss: 0.6695 - accuracy: 0.6249 - val_loss: 0.5926 - val_accuracy: 0.7359
Epoch 2/50
80/80 [==============================] - 0s 1ms/step - loss: 0.5992 - accuracy: 0.7237 - val_loss: 0.5649 - val_accuracy: 0.7359
Epoch 3/50
80/80 [==============================] - 0s 1ms/step - loss: 0.5551 - accuracy: 0.7511 - val_loss: 0.5511 - val_accuracy: 0.7516
Epoch 4/50
80/80 [==============================] - 0s 1ms/step - loss: 0.5419 - accuracy: 0.7507 - val_loss: 0.5411 - val_accuracy: 0.7641

/Users/jurgentoth/opt/anaconda3/envs/dl/lib/python3.8/site-packages/tensorflow/python/keras/engine/functional.py:540: UserWarning: Input dict contained keys ['id'] which did not match any model input. They will be ignored by the model.
  warnings.warn(


## Inference on new data

In [24]:
sample = {
    "acceloX": 10,
    "acceloY": -11,
    "acceloZ": 1.8,
    "userAcceloX": 0.48,
    "userAcceloY": 0.73,
    "userAcceloZ": 3.04,
    "gyroX": 5.1,
    "gyroY": -3.1,
    "gyroZ": 1.79,
    "lightSensor": 8.0
}

input_dict = {name: tf.convert_to_tensor([value]) for name, value in sample.items()}
predictions = model.predict(input_dict)
print("Likelihood that smartphone is unlocked:", predictions[0][0])


Likelihood that smartphone is unlocked: 0.7521107
